[INDEX](Kubernetes.ipynb)  [NEXT](3.pods.ipynb)

<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="images/containous.logo.png" width="100" /> </td></tr></table>

<img align="left" src="https://avatars1.githubusercontent.com/u/13629408?s=400&v=4" width="60" />
## 3. Running Pods, Deployments

<pre style="align:center" >
In this section we will work with a 3-node cluster comprised of 1 Master and 2 Workers.
</pre>
<center><img src="images/KubeNodes_1m_2w.svg.png" width="600" /></center>
    
<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="images/containous.logo.png" width="100" /> </td></tr></table>


    
<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="images/containous.logo.png" width="100" /> </td></tr></table>

# 3. Running Pods, Deployments

<pre style="align:center" >

**In this section** we will see how we can launch pods directly from the command-line
using 'kubectl run' and the resources this creates.

This is useful for experimentation but it is not declarative.

We will also see how we can create resources from a yaml file using the
'kubectl create' command, and how we can create several containers in a Pod.

We will look at how to scale Pods, and what happens when a Pod fails
</pre>
    
<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="images/containous.logo.png" width="100" /> </td></tr></table>

    
# Kubernetes Pods

## Useful functions

Here we define some useful functions to be able to run this notebook in isolation:

In [1]:
. ./.shell.functions
CLEANUP

In [2]:
kubectl get all

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   2d


## Cleanup

Let's first verify that we have no Pods, Deployments, Services running in the default namespace by running the '*kubectl get all' command

Then delete any running items (except '*svc/kubernetes*') if necessary

In [3]:
kubectl get all

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   2d


In [4]:
kubectl get all

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   2d


## Deploying Pods via 'kubectl run'
<!--We've seen how we can interrogate our cluster server using the kubectl tool, obtaining information about Resources such as Pods, Nodes in the system.-->
<pre style="align:center" >
Now let's create our own pods using 'kubectl run'.
We will see that this actually creates a Deployment and a ReplicaSet, as the example below:
<img src="images/KubernetesArchi_Deploy.svg.png" width="800" />

<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="images/containous.logo.png" width="100" /> </td></tr></table>

Normally we would not create pods directly using command-line arguments, as we will do here.

We would create objects in a repeatable, declarative way from yaml files, but we will do that later.

The below ```kubectl run``` command will create 2 pods (replicas), each running just 1 container from the k8s-demo image.

Note that we can specify labels on any kubernetes objects, using the -l option followd by key=value pairs, e.g.
    -l "key1=val1,key2=val2"
     
These labels are extremely useful allowing us to select a group of objects using a label selector.

Labels might be used to mark components of the cluster as dev, test or production for example.

Or they might be used to signal hardware specifics of a node, to distinguish physical nodes with SSD disks, or GPU which could then be prioritized for particular types of processing which might be disk i/o or CPU intensive.

k8sdemo is a small Docker image of 6MBy on the Docker hub [HERE](https://hub.docker.com/r/mjbright/k8s-demo/tags/)

<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="images/containous.logo.png" width="100" /> </td></tr></table>


We want to see what Resources are created, so let's just check again what exists already ... by running 'kubectl get all'

<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="images/containous.logo.png" width="100" /> </td></tr></table>


In [5]:
kubectl get all

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   2d


Next we will run 'kubectl run' and immediately run 'kubectl get all' again.

In [6]:
kubectl run k8s-demo -l "app=k8s-demo" --image=mjbright/k8s-demo:1 --replicas=2 --port=8080
kubectl get all

deployment "k8s-demo" created
NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            0           1s

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-6b76bd84d7   2         2         0         1s

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            0           1s

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-6b76bd84d7   2         2         0         1s

NAME                           READY     STATUS              RESTARTS   AGE
po/k8s-demo-6b76bd84d7-fpt4k   0/1       ContainerCreating   0          1s
po/k8s-demo-6b76bd84d7-s5t2p   0/1       ContainerCreating   0          1s

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   2d


In the above output we see that the only running resource initially was an internal service.

Once we perform '''kubectl run'' this spawns a **deployment** called *k8s-demo*, which creates a corrsponding **replicaset** which in turn spawns 2 **pods**.

As we performed the '''kubectl get all''' immediately after the run, we see the status before the pod containers have been created ... in fact there is a delay whilst the images are downloaded (for a first run) and pods instantiated.

Re-running '''kubectl get all''' shows the pods as *running* now.

So our 'kubectl run' has created several resources as shown below:
![](images/KubernetesArchi_Deploy_k8sdemo1.svg.png)

In [7]:
untilState pod k8s-demo Running -q &&
    kubectl get pods

NAME                        READY     STATUS    RESTARTS   AGE
k8s-demo-6b76bd84d7-fpt4k   1/1       Running   0          6s
k8s-demo-6b76bd84d7-s5t2p   1/1       Running   0          6s


... with more information, using option '*-o wide*'

In [8]:
kubectl get pods -o wide

NAME                        READY     STATUS    RESTARTS   AGE       IP           NODE
k8s-demo-6b76bd84d7-fpt4k   1/1       Running   0          6s        10.192.2.6   kube-node-1
k8s-demo-6b76bd84d7-s5t2p   1/1       Running   0          6s        10.192.3.6   kube-node-2


## Accessing our application

To access our application pods we should expose them as a service.
This would have the advantage of being able to address a virtual ip address for our service as pods themselves come and go (so we don't need to determine their address), and also allows load balancing between pods.

### Accessing our application - directly accessing Pods

But there are other more basic ways that we will investigate first.

If we perform '''kubectl get pods -o wide''' we will see the cluster ip of our pods, enabling us to access directly to the pods.

In [9]:
kubectl get pods -o wide 

NAME                        READY     STATUS    RESTARTS   AGE       IP           NODE
k8s-demo-6b76bd84d7-fpt4k   1/1       Running   0          7s        10.192.2.6   kube-node-1
k8s-demo-6b76bd84d7-s5t2p   1/1       Running   0          7s        10.192.3.6   kube-node-2


In [10]:
POD1=$(kubectl get pod -o wide --no-headers | grep k8s-demo | head -1 | awk '{print $1;}')
POD2=$(kubectl get pod -o wide --no-headers | grep k8s-demo | tail -1 | awk '{print $1;}')
POD1_IP=$(kubectl get pod -o wide --no-headers | grep k8s-demo | head -1 | awk '{print $6;}')
POD2_IP=$(kubectl get pod -o wide --no-headers | grep k8s-demo | tail -1 | awk '{print $6;}')

echo POD1=$POD1 POD1_IP=$POD1_IP
echo POD2=$POD2 POD2_IP=$POD2_IP

POD1=k8s-demo-6b76bd84d7-fpt4k POD1_IP=10.192.2.6
POD2=k8s-demo-6b76bd84d7-s5t2p POD2_IP=10.192.3.6


Let's look at the yaml information for POD1:

In [11]:
kubectl get pod $POD1 -o yaml

apiVersion: v1
kind: Pod
metadata:
  creationTimestamp: 2018-03-22T09:52:14Z
  generateName: k8s-demo-6b76bd84d7-
  labels:
    app: k8s-demo
    pod-template-hash: "2632684083"
  name: k8s-demo-6b76bd84d7-fpt4k
  namespace: default
  ownerReferences:
  - apiVersion: extensions/v1beta1
    blockOwnerDeletion: true
    controller: true
    kind: ReplicaSet
    name: k8s-demo-6b76bd84d7
    uid: b30eb5f3-2db6-11e8-ad88-02420ac00002
  resourceVersion: "281435"
  selfLink: /api/v1/namespaces/default/pods/k8s-demo-6b76bd84d7-fpt4k
  uid: b3109c58-2db6-11e8-ad88-02420ac00002
spec:
  containers:
  - image: mjbright/k8s-demo:1
    imagePullPolicy: IfNotPresent
    name: k8s-demo
    ports:
    - containerPort: 8080
      protocol: TCP
    resources: {}
    terminationMessagePath: /dev/termination-log
    terminationMessagePolicy: File
    volumeMounts:
    - mountPath: /var/run/secrets/kubernetes.io/serviceaccount
      name: default-token-797bq
      readOnly: true
  dnsPolicy: ClusterFirst
 

We can see that the containerPort 8080/TCP is exposed (as we specified on our initial '*kubectl run*' command).

So now we can access to the pods directly on their port 8080.

In [12]:
curl http://$POD1_IP:8080




                               
                    (((((((((                  
               .(((((((((((((((((.             
           .((((((((((((&((((((((((((.         
       /((((((((((((((((@((((((((((((((((/     
      ((((((((((((((((((@((((((((((((((((((    
     *(((((##((((((@@@@@@@@@@@((((((%#(((((*   
     (((((((@@@(@@@@#((@@@((#@@@@(@@@(((((((   
    *(((((((((@@@@(((((@@@(((((@@@@(((((((((,  
    (((((((((@@@%@@@@((@@@((@@@@%@@@(((((((((  
   .(((((((((@@((((@@@@@@@@@@@((((@@(((((((((. 
   (((((((((&@@(((((@@@(((@@@(((((@@&((((((((( 
   (((((((((&@@@@@@@@@@#(#@@@@@@@@@@&((((((((( 
  ((((((@@@@@@@@(((((@@@@@@@(((((&@@@@@@@((((((
  (((((((((((%@@((((%@@@(@@@%((((@@&(((((((((((
   ((((((((((((@@@((@@%(((%@@((@@@(((((((((((( 
     (((((((((((#@@@@%(((((&@@@@#(((((((((((   
      /(((((((((((@@@@@@@@@@@@@(((((((((((/    
        (((((((((@@(((((((((((@@(((((((((      
          (((((((&(((((((((((((&(((((((        
           /(((((((((((((((((((((((((/         
     

In [13]:
curl http://$POD2_IP:8080




                               
                    (((((((((                  
               .(((((((((((((((((.             
           .((((((((((((&((((((((((((.         
       /((((((((((((((((@((((((((((((((((/     
      ((((((((((((((((((@((((((((((((((((((    
     *(((((##((((((@@@@@@@@@@@((((((%#(((((*   
     (((((((@@@(@@@@#((@@@((#@@@@(@@@(((((((   
    *(((((((((@@@@(((((@@@(((((@@@@(((((((((,  
    (((((((((@@@%@@@@((@@@((@@@@%@@@(((((((((  
   .(((((((((@@((((@@@@@@@@@@@((((@@(((((((((. 
   (((((((((&@@(((((@@@(((@@@(((((@@&((((((((( 
   (((((((((&@@@@@@@@@@#(#@@@@@@@@@@&((((((((( 
  ((((((@@@@@@@@(((((@@@@@@@(((((&@@@@@@@((((((
  (((((((((((%@@((((%@@@(@@@%((((@@&(((((((((((
   ((((((((((((@@@((@@%(((%@@((@@@(((((((((((( 
     (((((((((((#@@@@%(((((&@@@@#(((((((((((   
      /(((((((((((@@@@@@@@@@@@@(((((((((((/    
        (((((((((@@(((((((((((@@(((((((((      
          (((((((&(((((((((((((&(((((((        
           /(((((((((((((((((((((((((/         
     

## What happens if a pod dies?

In [14]:
kubectl get pods -o wide

NAME                        READY     STATUS    RESTARTS   AGE       IP           NODE
k8s-demo-6b76bd84d7-fpt4k   1/1       Running   0          9s        10.192.2.6   kube-node-1
k8s-demo-6b76bd84d7-s5t2p   1/1       Running   0          9s        10.192.3.6   kube-node-2


<!-- When we deleted the 2-containers-in-a-pod Pod it just died. -->

If we delete one of our k8s-demo Pods we will see that a new Pod is created automatically, this is because it is managed by a ReplicaSet.

Running a Pod via "kubectl run" creates a Deployment which then creates a ReplicaSet.

Let's kill one of our Pods and see what happens


In [15]:
kubectl get pod | grep k8s-demo | head -1 | awk '{ print $1;}'

POD1=$(kubectl get pod | grep k8s-demo | head -1 | awk '{ print $1;}')

k8s-demo-6b76bd84d7-fpt4k


In [16]:
kubectl delete pod $POD1

pod "k8s-demo-6b76bd84d7-fpt4k" deleted


In [17]:
kubectl get pods -o wide

NAME                        READY     STATUS              RESTARTS   AGE       IP           NODE
k8s-demo-6b76bd84d7-4z7mm   0/1       ContainerCreating   0          0s        <none>       kube-node-1
k8s-demo-6b76bd84d7-fpt4k   1/1       Terminating         0          10s       10.192.2.6   kube-node-1
k8s-demo-6b76bd84d7-s5t2p   1/1       Running             0          10s       10.192.3.6   kube-node-2


In [18]:
sleep 10; kubectl get pods -o wide

NAME                        READY     STATUS    RESTARTS   AGE       IP           NODE
k8s-demo-6b76bd84d7-4z7mm   1/1       Running   0          11s       10.192.2.7   kube-node-1
k8s-demo-6b76bd84d7-s5t2p   1/1       Running   0          21s       10.192.3.6   kube-node-2


We see that our Pod was deleted but that a new one was created, this is what we expect when our Pod is managed by a ReplicaSet.

[INDEX](Kubernetes.ipynb)  [NEXT](3.pods.ipynb)

**Note:** In a real deployment of course the actual Pod ips would not be externally accessible so we would want some way of making the nodes available on a well known address.


<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="images/containous.logo.png" width="100" /> </td></tr></table>